In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm, colors, rcParams

import numpy as np

import bayesmark.constants as cc
from bayesmark.path_util import abspath
from bayesmark.serialize import XRSerializer
from bayesmark.constants import ITER, METHOD, TEST_CASE, OBJECTIVE, VISIBLE_TO_OPT

In [ ]:
# User settings, must specify location of the data to make plots here for this to run
DB_ROOT = abspath(".")
DBID = "bo_example_folder"
metric_for_scoring = VISIBLE_TO_OPT

In [ ]:
# Matplotlib setup
# Note this will put type-3 font BS in the pdfs, if it matters
rcParams["mathtext.fontset"] = "stix"
rcParams["font.family"] = "STIXGeneral"

In [ ]:
def build_color_dict(names):
    """Make a color dictionary to give each name a mpl color.
    """
    norm = colors.Normalize(vmin=0, vmax=1)
    m = cm.ScalarMappable(norm, cm.tab20)
    color_dict = m.to_rgba(np.linspace(0, 1, len(names)))
    color_dict = dict(zip(names, color_dict))
    return color_dict

In [ ]:
# Load the data
agg_results_ds, meta = XRSerializer.load_derived(DB_ROOT, db=DBID, key=cc.PERF_RESULTS)

In [ ]:
# Setup for plotting
method_list = agg_results_ds.coords[METHOD].values
method_to_rgba = build_color_dict(method_list.tolist())

In [ ]:
# Make the plots for inidividual test functions
for func_name in agg_results_ds.coords[TEST_CASE].values:
    plt.figure(figsize=(5, 5), dpi=300)
    for method_name in method_list:
        curr_ds = agg_results_ds.sel({TEST_CASE: func_name, METHOD: method_name, OBJECTIVE: metric_for_scoring})

        plt.fill_between(
            curr_ds.coords[ITER].values,
            curr_ds[cc.LB_MED].values,
            curr_ds[cc.UB_MED].values,
            color=method_to_rgba[method_name],
            alpha=0.5,
        )
        plt.plot(
            curr_ds.coords[ITER].values,
            curr_ds[cc.PERF_MED].values,
            color=method_to_rgba[method_name],
            label=method_name,
            marker=".",
        )
    plt.xlabel("evaluation", fontsize=10)
    plt.ylabel("median score", fontsize=10)
    plt.title(func_name)
    plt.legend(fontsize=8, bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0.0)
    plt.grid()

    plt.figure(figsize=(5, 5), dpi=300)
    for method_name in method_list:
        curr_ds = agg_results_ds.sel({TEST_CASE: func_name, METHOD: method_name, OBJECTIVE: metric_for_scoring})

        plt.fill_between(
            curr_ds.coords[ITER].values,
            curr_ds[cc.LB_MEAN].values,
            curr_ds[cc.UB_MEAN].values,
            color=method_to_rgba[method_name],
            alpha=0.5,
        )
        plt.plot(
            curr_ds.coords[ITER].values,
            curr_ds[cc.PERF_MEAN].values,
            color=method_to_rgba[method_name],
            label=method_name,
            marker=".",
        )
    plt.xlabel("evaluation", fontsize=10)
    plt.ylabel("mean score", fontsize=10)
    plt.title(func_name)
    plt.legend(fontsize=8, bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0.0)
    plt.grid()